# **線性迴歸的成本函數**

## **目標**
在本實驗中，您將：
- 實現並探索單變量線性迴歸的`成本`函數。

<br>

## **工具**
在本實驗中，您將使用：

- NumPy，一個用於科學計算的流行庫

- Matplotlib，一個用於繪製數據的流行庫

- 本地目錄中lab_utils_uni.py文件中的本地繪圖程序

In [ ]:
import sys, os
import numpy as np
try:
    %matplotlib widget
except:
    %matplotlib inline
    print("Colab not support matplotlib widget")
import matplotlib.pyplot as plt
from pathlib import Path

#region 匯入模組
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common.py", f"{UTILS_DIR}/lab_utils_common.py")
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_uni.py", f"{UTILS_DIR}/lab_utils_uni.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_uni import plt_intuition, plt_stationary, plt_update_onclick, soup_bowl
plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion 匯入模組

<br>

## **問題說明**

您希望有一個模型可以根據房屋的大小預測房價。讓我們使用與前一個實驗相同的兩個數據點:

* 一棟 1000 坪的房屋售價為 $300,0000

* 一棟 2000 坪的房屋售價為 $500,0000

<div align="center">

| 大小 (1000 坪)     | 價格 (萬) |
| -------------------| ------------------------ |
| 1.0                 | 300.0                      |
| 2.0                  | 500.0                      |

</div>

請完成下面程式碼區塊，以建立你的 `x_train` 和 `y_train` 變數。

In [ ]:
x_train = np.array([1.0, 2.0])           #(size in 1000 ping)
y_train = np.array([300.0, 500.0])       #(price in 10000s of dollars)

<br>

## **計算成本 (Computing Cost)**
本次實驗中， `成本` 這個詞可能令人有些混淆。在這裡:

- `成本` 指的是衡量我們的模型預測房價準確程度的指標，也就是「模型預測與實際價格之間的誤差大小」

- `價格` 這個詞，則專門用來指房屋本身的售價。

單變量成本方程式為：
  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{1}$$ 
 
其中 
  $$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{2}$$
  
- $f_{w,b}(x^{(i)})$ 是我們使用參數 $w,b$ 對樣本 $i$ 的預測。 

- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ 是目標值與預測值之間的平方差。 

- 這些差異在所有 $m$ 個示例上求和並除以 `2m` 得到成本 $J(w,b)$。 
 
>注意，在課堂講義中，求和範圍顯示是從1到m，而程式碼中則是從0到m-1。

請定義並完成 `compute_cost(x, y, w, b)` 函式於下方，

其中，此計算成本函式會透過逐一遍歷每個樣本，來計算成本。在每一次迴圈中，它會執行以下步驟：

- 計算預測值 `f_wb`

- 計算目標值與預測值之間的差異並平方。

- 將其添加到總成本中。

In [ ]:
def compute_cost(x, y, w, b): 
    """
    計算線性迴歸的成本函數。
    
    參數:
      x (ndarray (m,)): 數據，m個示例 
      y (ndarray (m,)): 目標值
      w,b (標量)    : 模型參數  
    
    返回
        total_cost (浮點數): 使用w,b作為線性迴歸參數
               擬合x和y中數據點的成本
    """
    # 訓練示例數量
    m = x.shape[0] 
    
    cost_sum = 0 
    for i in range(m): 
        f_wb = w * x[i] + b   
        cost = (f_wb - y[i]) ** 2  
        cost_sum += cost  
        
    total_cost = (1 / (2 * m)) * cost_sum  

    return total_cost

<br>

## **成本函數直觀理解**

你的目標是找到一個模型 $f_{w,b}(x) = wx + b$ 中的參數 $w,b$，能夠準確地根據輸入 $x$ 預測房屋價格。

而成本是衡量模型在訓練數據上準確性的指標。

根據前面提到的成本函數公式 (1)，如果選擇合適的 $w$ 和 $b$ ，使模型預測值 $f_{w,b}(x)$ 與真實值 $y$ 相匹配，則 $(f_{w,b}(x^{(i)}) - y^{(i)})^2 $ 項將趨近於0，成本函數達到最小化。

在這個只有兩筆資料的簡單例子中，你實際上可以讓模型完全擬合資料!

在前一個實驗中，我們已經發現 $b=100$ 是最佳截距值，因此接下來我們固定 $b=100$ ，只需專注於調整 $w$

<br/>

下面，使用滑塊控制選擇使成本最小化的 $w$ 值。圖表更新可能需要幾秒鐘。

In [ ]:
plt.close('all') 
plt_intuition(x_train,y_train)

該圖包含幾個值得一提的點:

- 當 $w = 200$ 時，成本最小化，這與前一個實驗的結果相符。

- 由於在成本方程式中目標與預測之間的差異是平方的，當 $w$ 太大或太小時，成本會迅速增加。

- 使用通過最小化成本選擇的 `w` 和 `b` ，得到的直線可以與資料完美匹配。

<br>

## **3D可視化成本函數**

你可以通過3D繪圖或等高線圖來觀察成本是如何隨著 `w` 和 `b` 兩者的變化而改變。 

這樣可以幫助你更直觀地理解「成本函數曲面」的形狀 —— 通常是一個 **碗狀(convex bowl shape)** 的圖形

值得注意的是，本課程中有些繪圖部分會稍微複雜一點。

但不用擔心，這些繪圖函式已經在實驗室開始時匯入(一定要先執行上面程式)，所以請安心服用!

<br>

### **更大的數據集**
觀察包含更多資料點的情境是很有幫助的。這個資料集包含了不在同一直線上的資料點。這種情況下，我們是否還能找到使成本趨近 0 的 $w$ 和 $b$ 嗎？

In [ ]:
x_train = np.array([1.0, 1.7, 2.0, 2.5, 3.0, 3.2])
y_train = np.array([250, 300, 480,  430,   630, 730,])

在等高線圖中，嘗試點擊一個點來選擇 `w` 和 `b` ，以達到最低成本。

In [ ]:
plt.close('all') 
fig, ax, dyn_items = plt_stationary(x_train, y_train)
updater = plt_update_onclick(fig, ax, x_train, y_train, dyn_items)

如上圖左邊的虛線所示，這些虛線代表了訓練集中每個樣本對整體成本（cost）所貢獻的部分。在這個例子中，大約當 $w=209$ 且 $b=2.4$ 時，可以得到較低的成本值。

需要注意的是，因為訓練資料點並不在同一直線上，因此即使在最佳的 $w$ 和 $b$ 參數下，成本也無法降到 0

<br>

### **凸成本曲面**

成本函數因為誤差平方的這個特性，確保了整個「誤差曲面（error surface）」呈現凸形（convex），就像一個湯碗一樣。

這也代表著它一定會有一個全域最小值（global minimum）。

而我們可以透過在所有維度上 **沿著梯度方向下降（gradient descent）** 來抵達這個最低點。

在前一張圖中，由於 $w$ 和 $b$ 的尺度不同，讓這個「碗狀」的結構不太容易直觀看出。

因此，接下來這張圖（課堂中展示過的版本）讓 $w$ 和 $b$ 的尺度對稱化（symmetric scaling），

就能更清楚地看見那個典型的凸型誤差曲面形狀：

In [ ]:
soup_bowl()

<br>

# **恭喜！**

您已經學習了以下內容：

 - **成本函數 (Cost Function)** 

    - 提供了一個衡量您的預測與訓練數據匹配程度的指標。

 - **最小化成本（Minimizing the Cost)**

    - 可以幫助我們找到最佳的模型參數 $w$, $b$，使模型在訓練資料上達到最準確、最穩定的預測效果。

<br>